In [1]:
import codecs
import os
from collections import defaultdict
import pickle
import numpy as np

import pandas as pd

In [11]:
def run(file, csid, cusr, cans, sort_by):
    dirname = os.path.dirname("./")
#     dirname = os.path.dirname(__file__)
    infname = os.path.join(dirname, f'raw_input/{file}.csv')
    with codecs.open(infname, 'r', 'utf-8', 'ignore') as f:
        df = pd.read_csv(f)
        print(df.columns)
    print(df['CustomerNumber'].value_counts())

In [ ]:
run('Asteria_Eng_ver5', 'SkillID', 'CustomerNumber', 'AnswerResult', 'LearningDateTime')

In [2]:
file = 'Asteria_Eng_ver5'

dirname = os.path.dirname("./")
infname = os.path.join(dirname, f'raw_input/{file}.csv')
with codecs.open(infname, 'r', 'utf-8', 'ignore') as f:
    df = pd.read_csv(f)
    print(df.columns)

Index(['Unnamed: 0', 'SEQ', 'CustomerNumber', 'LearningDateTime',
       'ResourceNumber', 'SkillID', 'AnswerResult'],
      dtype='object')


In [27]:
skill_counts = df['SkillID'].value_counts()

In [28]:
skill_remove = skill_counts[skill_counts <= 200].index

In [29]:
df.replace(skill_remove, np.nan, inplace=True)

In [16]:
len(df)

724052

In [3]:
# scs = dict()
# for i, row in df.iterrows():
#     skill_category = row['SkillID'][:3]
#     if skill_category not in scs:
#         scs[skill_category] = 1
#     else:
#         scs[skill_category] += 1
        
# print(scs)

# set(df['SkillID'].map(lambda x: x[:3]))
# => {'NEL', 'NER', 'NES', 'NEW'}
skill_cats = {'NEL', 'NER', 'NES', 'NEW'}
for skill_cat in skill_cats:
    sc_df = df[df['SkillID'].map(lambda x: x[:3]) == skill_cat]
    print(len(sc_df))

139275
165070
279035
140672


In [4]:
def id_generator():
    n = 0
    while True:
        yield n
        n += 1


file, csid, cusr, cans, sort_by = 'Asteria_Eng_ver5', 'SkillID', 'CustomerNumber', 'AnswerResult', 'LearningDateTime'
outfile_name=''

dirname = os.path.dirname('./')
infname = os.path.join(dirname, f'raw_input/{file}.csv')

with codecs.open(infname, 'r', 'utf-8', 'ignore') as f:
    df = pd.read_csv(f)
    print(df.columns)

for skill_cat in {'NEL', 'NER', 'NES', 'NEW'}:
    outfname = os.path.join(dirname, f'input/{outfile_name if outfile_name else file}_{skill_cat}.pickle')
    outdicname = os.path.join(dirname, f'input/{outfile_name if outfile_name else file}_{skill_cat}_dic.pickle')
    # order_id,assignment_id,user_id,assistment_id,problem_id,original,correct,attempt_count,ms_first_response,tutor_mode,answer_type,sequence_id,student_class_id,position,type,base_sequence_id,skill_id,skill_name,teacher_id,school_id,hint_count,hint_total,overlap_time,template_id,answer_id,answer_text,first_action,bottom_hint,opportunity,opportunity_original
    print(outfname)

    if sort_by:
        df = df.sort_values(by=sort_by)

    it = iter(id_generator())

    processed = defaultdict(list)
    problems = defaultdict(lambda: next(it))

    sc_df = df[df['SkillID'].map(lambda x: x[:3]) == skill_cat]
    for idx, row in sc_df.iterrows():
        # nanは無視する
        sid = row[csid]
        usr = row[cusr]
        ans = row[cans]

        if pd.isnull(sid) or pd.isnull(ans) or pd.isnull(usr):
            continue
        # processed[row.user_id].append((problems[row.problem_id], row.correct))
        processed[usr].append((problems[sid], ans))

    print('Skill Category:', skill_cat)
    print('Skills:', len(problems))
    print('Students:', len(processed))
    print('=====================')

    # Save processed data
    with open(outfname, 'wb') as f:
        pickle.dump(dict(processed), f)
    with open(outdicname, 'wb') as f:
        pickle.dump(dict(problems), f)

Index(['Unnamed: 0', 'SEQ', 'CustomerNumber', 'LearningDateTime',
       'ResourceNumber', 'SkillID', 'AnswerResult'],
      dtype='object')
./raw_input/Asteria_Eng_ver5.csv
Skill Category: NES
Skills: 66
Students: 1403
./raw_input/Asteria_Eng_ver5.csv
Skill Category: NEL
Skills: 50
Students: 1921
./raw_input/Asteria_Eng_ver5.csv
Skill Category: NEW
Skills: 163
Students: 1593
./raw_input/Asteria_Eng_ver5.csv
Skill Category: NER
Skills: 47
Students: 1804
